In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer#,TfidfTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import os

In [ ]:
trainc=pd.read_csv('data1/train.csv')
trainc=trainc[trainc.label!=-1]

testc=pd.read_csv('data1/test.csv')
testc['label'] = -1

data=pd.concat([trainc,testc])
data=data.fillna('-1')

del trainc
del testc

In [ ]:
data.info()

In [ ]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','house']
vector_feature=['appIdAction','appIdInstall','marriageStatus','interest1','interest2','interest3','interest4','interest5','kw1','kw2','kw3','topic1','topic2','topic3']
#LabelEncoder将各种标签分配一个可数的连续编号
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
        
train=data[data.label!=-1]
train_y=train.pop('label')

test=data[data.label==-1]
res=test[['aid','uid']]
test=test.drop('label',axis=1)

train_x=train[['advertiserId','campaignId', 'creativeId','creativeSize','adCategoryId', 'productId','productType']]
              # ,'ct_0','ct_1','ct_2','ct_3','ct_4','os_0','os_1','os_2']]
              # ,'has_appins','has_appact','max_marriageStatus']]
test_x = test[['advertiserId','campaignId', 'creativeId','creativeSize','adCategoryId', 'productId', 'productType']]
             # ,'ct_0','ct_1','ct_2','ct_3','ct_4','os_0','os_1','os_2']]
              ##,'has_appins','has_appact','max_marriageStatus']]

In [ ]:
#one-hot特征reshape
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('one-hot prepared !')
del train_a
del test_a

In [ ]:
##向量特征reshape
cv=CountVectorizer(ngram_range=(1, 2),token_pattern='(?u)\\b\\w+\\b')
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
print('cv prepared !')
del train_a
del test_a

In [ ]:
def LGB_predict(train_x,train_y,test_x,res):
    print("LGB test")
    clf = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,zero_as_missing=True,
        max_depth=-1, n_estimators=1500, objective='binary',
        subsample=0.9, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.2, min_child_weight=50, random_state=2018, n_jobs=100
    )
    clf.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='auc',early_stopping_rounds=100)
    res['score'] = clf.predict_proba(test_x)[:,1]
    res['score'] = res['score'].apply(lambda x: float('%.6f' % x))
    res.to_csv('../data/submission.csv', index=False)
    os.system('zip baseline.zip ../data/submission.csv')
    return clf

model=LGB_predict(train_x,train_y,test_x,res)